In [165]:
import numpy as np 
import pandas as pd
import torch
import tiktoken 

In [48]:
x = np.array([[1,2,3]])
print(x.shape)
x

(1, 3)


array([[1, 2, 3]])

In [49]:
y = np.array([[1],[2],[3]])
print(y.shape)
y

(3, 1)


array([[1],
       [2],
       [3]])

In [50]:
x + y

array([[2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [53]:
torch.__version__

'2.3.1'

In [54]:
torch.cuda.is_available()

True

In [83]:
# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])

# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [84]:
tensor2d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [86]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [93]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [96]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [118]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
Total number of trainable model parameters: 2213


In [119]:
model.layers[0].weight

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)

In [120]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [122]:
torch.manual_seed(123)

X = torch.rand((1, 50))
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [123]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [124]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [131]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


In [153]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)

#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # considered pytorch best practice
model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
epochs = 3

for epoch in range(epochs):

    model.train()

    for batch_idx, (features,labels) in enumerate(train_loader):
        features,labels = features.to(device), labels.to(device)

        logits = model(features)
        loss = F.cross_entropy(logits,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train/Val Loss: {loss:.2f}")
      
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [143]:
# can make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train) # logits
print(outputs)

torch.set_printoptions(sci_mode=False) 
probs = torch.softmax(outputs,dim=1) # class probas
print(probs)

preds = torch.argmax(probs,dim=1) # convert probas to class preds
print(preds)
 
preds = torch.argmax(outputs,dim=1) # could also get preds direct from logits
print(preds)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])
tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])
tensor([0, 0, 0, 1, 1])


In [146]:
(torch.sum(preds == y_train) / len(y_train))*100 # acc

tensor(100.)

In [148]:
for key in model.state_dict(): 
    print(key)

layers.0.weight
layers.0.bias
layers.2.weight
layers.2.bias
layers.4.weight
layers.4.bias


In [149]:
torch.save(model.state_dict(),"model_test.pth")

In [150]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model_test.pth", weights_only=True))

<All keys matched successfully>

In [ ]:
#model = MyTransformerDecoderModel(...)
#model = model.half().cuda()  # Use float16 to save VRAM

# Check memory usage
#print(torch.cuda.memory_allocated() / 1024 ** 2, "MB allocated")

In [163]:
x = torch.randn(2,2,3)
y = torch.randn(2,2,3)
y = y.view()
print(x)
print(x.shape)
print("")
print(y)
print(y.shape)

tensor([[[ 1.6782,  0.0878,  1.1767],
         [-0.1191,  0.2469,  0.6126]],

        [[-0.3501,  0.2827, -0.0036],
         [ 0.7141, -1.0415,  0.0298]]])
torch.Size([2, 2, 3])

tensor([[[ 1.7629,  0.0267,  0.6328],
         [ 0.0105,  1.2284, -0.4705]],

        [[-0.5538, -0.2770, -0.9325],
         [-0.5085, -0.0058, -0.4462]]])
torch.Size([2, 2, 3])


In [170]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
print(f"Vocab size: {vocab_size}")
es = enc.encode("Hello, Marcus Aurelius.")
print(es)
ds = enc.decode(es)
print(ds)

Vocab size: 50257
[15496, 11, 17068, 43737, 75, 3754, 13]
Hello, Marcus Aurelius.


In [301]:
# optimal mha
# no loops, one big matrix 
B = 2
T = 2
d = 3
hd = 3 # in practice it will be d // n_heads
nh = 3 # num heads

#X = torch.randint(low=1,high=9,size=(B,T,d))
X = torch.randn((B,T,d))

In [302]:
ll = torch.nn.Linear(d,3*hd*nh,bias=True)
QKV = ll(X)
print(QKV)
QKV.shape

tensor([[[-0.6271,  0.5224,  0.1951,  0.3762,  0.1657, -0.0828, -0.4390,
          -0.6100, -0.5298, -0.5523, -0.0427, -0.0018,  0.4463, -0.1734,
          -0.4770, -0.3574,  0.4644,  0.3374,  0.2257, -0.3733,  0.0748,
          -0.1851,  0.2172,  0.0971,  0.3903, -0.0038,  0.6534],
         [-0.8362, -0.4417, -0.8353,  0.0167,  0.8215, -0.6877, -0.6109,
          -1.2496, -0.9900, -0.6647,  0.4168, -0.2292,  0.8116, -0.3229,
           0.3085,  0.3755,  0.0404,  0.7712,  0.3532, -0.4491,  0.3014,
           0.3019, -0.1068,  0.1887,  1.2488,  0.1411,  0.7244]],

        [[-0.6630,  1.1565,  0.9561,  0.4523, -0.3095,  0.4146, -0.5492,
          -0.3754, -0.3325, -0.5808, -0.1150,  0.0276,  0.0410, -0.2123,
          -0.8457, -0.8997,  0.6915, -0.1161,  0.1741, -0.3182, -0.1620,
          -0.2061,  0.2560, -0.1475, -0.2548,  0.2797,  0.8447],
         [-1.0123, -0.0897, -0.4756, -0.0057,  0.7449, -0.3089, -0.7296,
          -1.0994, -0.7219, -0.3114,  0.0220, -0.5110,  0.3338, -0.6215,


torch.Size([2, 2, 27])

In [303]:
print(QKV.view(B,T,nh,3,hd).shape)
qkv = QKV.view(B,T,nh,3,hd) 

torch.Size([2, 2, 3, 3, 3])


In [304]:
qkv[:,:,0,1,:] # K of head 1

tensor([[[ 0.3762,  0.1657, -0.0828],
         [ 0.0167,  0.8215, -0.6877]],

        [[ 0.4523, -0.3095,  0.4146],
         [-0.0057,  0.7449, -0.3089]]], grad_fn=<SliceBackward0>)

In [305]:
qkv[:,:,:,1,:] # All Ks

tensor([[[[ 0.3762,  0.1657, -0.0828],
          [ 0.4463, -0.1734, -0.4770],
          [-0.1851,  0.2172,  0.0971]],

         [[ 0.0167,  0.8215, -0.6877],
          [ 0.8116, -0.3229,  0.3085],
          [ 0.3019, -0.1068,  0.1887]]],


        [[[ 0.4523, -0.3095,  0.4146],
          [ 0.0410, -0.2123, -0.8457],
          [-0.2061,  0.2560, -0.1475]],

         [[-0.0057,  0.7449, -0.3089],
          [ 0.3338, -0.6215,  0.0346],
          [ 0.1250, -0.2847, -0.1004]]]], grad_fn=<SliceBackward0>)

In [306]:
# Q @ K.T for all heads
attn_matrix = qkv[:,:,:,0,:] @ qkv[:,:,:,1,:].transpose(-2,-1)
# [2, 2, 3, 4] @ [2, 2, 4, 3] = [2, 2, 3, 3]
# [B, T, nh, hd] @ [B, T, hd, nh] = [B, T, nh, nh] --> the attention matrix for each head of each sequence of each batch
print(attn_matrix)
print(attn_matrix.shape)

tensor([[[[-0.1655, -0.4635,  0.2485],
          [-0.2147, -0.2382,  0.0928],
          [ 0.0169,  0.1298, -0.1156]],

         [[ 0.1977, -0.7937, -0.3629],
          [ 0.4888, -0.7448, -0.2885],
          [-0.5703,  0.5247,  0.2115]]],


        [[[-0.2614, -1.0813,  0.2916],
          [-0.2156, -0.0228,  0.0862],
          [ 0.1101,  0.2117, -0.0934]],

         [[ 0.0858, -0.2987, -0.0533],
          [ 0.1760, -0.1353,  0.0061],
          [-0.1693,  0.2661,  0.0954]]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 2, 3, 3])


In [271]:
qkv[:,:,0,0,:] # Q of head 1 across batches

tensor([[[-0.1726, -0.5149,  0.1027,  0.4157],
         [ 1.1429, -0.8068, -1.0322, -0.0285]],

        [[ 1.1552, -0.7293, -1.1359,  0.1179],
         [ 0.8145, -1.0387, -1.1086,  0.8708]]], grad_fn=<SliceBackward0>)

In [273]:
qkv[:,:,1,0,:] # Q of head 2 across batches

tensor([[[ 0.7306,  0.6166,  0.1692, -0.8864],
         [ 1.4255,  0.9988, -0.5525,  0.1607]],

        [[ 1.0821,  0.6627, -0.3456,  0.1858],
         [ 0.1764, -0.4642,  0.7496, -0.0987]]], grad_fn=<SliceBackward0>)

In [254]:
QKV[:,:,0*hd:1*hd] # Q of head 1 across batches
#QKV[:,:,1*hd:2*hd] # K of head 1
#QKV[:,:,2*hd:3*hd] # V of head 1

tensor([[[-0.1726, -0.5149,  0.1027,  0.4157],
         [ 1.1429, -0.8068, -1.0322, -0.0285]],

        [[ 1.1552, -0.7293, -1.1359,  0.1179],
         [ 0.8145, -1.0387, -1.1086,  0.8708]]], grad_fn=<SliceBackward0>)

In [ ]:
QKV[:,:,3*hd:4*hd] # Q of head 2

tensor([[[ 0.7306,  0.6166,  0.1692, -0.8864],
         [ 1.4255,  0.9988, -0.5525,  0.1607]],

        [[ 1.0821,  0.6627, -0.3456,  0.1858],
         [ 0.1764, -0.4642,  0.7496, -0.0987]]], grad_fn=<SliceBackward0>)

In [238]:
# Q @ K.T
QKV[:,:,:hd] @ QKV[:,:,hd:2*hd].transpose(-2,-1)
# [B,T,hd] @ [B,hd,T] --> [B,T,T]

tensor([[[ 0.6610,  0.3774],
         [-1.3176,  0.1476]],

        [[ 1.1538,  1.1550],
         [ 0.7298,  0.6045]]], grad_fn=<UnsafeViewBackward0>)